### Extracting text from a PDF file and splitting that text into chunks 

In [1]:
pip install -U langchain-pymupdf4llm


  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pymupdf-1.25.5-cp39-abi3-macosx_11_0_arm64.whl.metadata (3.4 kB)
  Using cached jsonpointer-3.0.0-py2.py3-none-any.whl.metadata (2.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.8 MB/s eta 0:00:00a 0:00:01
Using cached pymupdf-1.25.5-cp39-abi3-macosx_11_0_arm64.whl (18.6 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached httpx-0.28.1-py3-none-

In [1]:
import fitz  # PyMuPDF
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document


In [2]:
uploaded_file = "/Users/arunekambaram/Desktop/RAG-ChatBot/data/GPT-4_VS_Human_translators (1).pdf"


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(uploaded_file)
documents = loader.load()
text = "\n".join([doc.page_content for doc in documents])


In [ ]:
# loader = PyMuPDF4LLMLoader(
#    uploaded_file,
#     mode="page",
# )
# docs = loader.load()

# print(len(docs))
# pprint.pp(docs[0].metadata)

NameError: name 'PyMuPDF4LLMLoader' is not defined

In [4]:
# Adaptive chunking
splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        separators=["\n\n", "\n", ".", " "]
    )

In [5]:
chunks = splitter.create_documents([text])
chunks

[Document(page_content='GPT-4 vs. Human Translators: A Comprehensive Evaluation of\nTranslation Quality Across Languages, Domains, and Expertise Levels\nJianhao Yan1,2∗\nPingchuan Yan3∗\nYulong Chen4∗\nJudy Li5\nXianchao Zhu5\nYue Zhang2,6,\x001 Zhejiang University\n2 School of Engineering, Westlake University\n3 University College London\n4 University of Cambridge\n5 Lan-Bridge Group\n6 Institute of Advanced Technology, Westlake Institute for Advanced Study\nelliottyan37@gmail.com\nAbstract\nThis study comprehensively evaluates the\ntranslation quality of Large Language Mod-\nels (LLMs), specifically GPT-4, against hu-\nman translators of varying expertise lev-\nels across multiple language pairs and do-\nmains. Through carefully designed annota-\ntion rounds, we find that GPT-4 performs\ncomparably to junior translators in terms of'),
 Document(page_content='mains. Through carefully designed annota-\ntion rounds, we find that GPT-4 performs\ncomparably to junior translators in terms 

In [15]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers import EnsembleRetriever
from langchain.schema import Document

In [16]:
# Simple hybrid retriever setup
from typing import List

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/var/folders/4v/zjjpj02j0f59s5fjqq0twgp40000gn/T/ipykernel_23285/1836249673.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/Users/arunekambaram/Desktop/RAG-ChatBot/venv/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [25]:
vectorstore = FAISS.from_documents(chunks, embedding_model)

In [18]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x367e70fa0>, search_kwargs={'k': 4})

In [19]:
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


In [26]:
template = """
    You are a helpful assistant. Use only the provided context to answer the question.
    If the answer is not in the context, say "The document does not contain that information."

    Context:
    {context}

    Question:
    {question}
    """

In [28]:
prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )

In [22]:
llm = Ollama(model="llama3")  # or "mixtral"


In [23]:
qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt},
        return_source_documents=False
)


In [24]:
qa_chain

RetrievalQA(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='\n    You are a helpful assistant. Use only the provided context to answer the question.\n    If the answer is not in the context, say "The document does not contain that information."\n\n    Context:\n    {context}\n\n    Question:\n    {question}\n    '), llm=Ollama(model='llama3')), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x367e70fa0>, search_kwargs={'k': 4}))

In [34]:
query = "The average win rate of GPT"
response = qa_chain.run(query)

print(response)

The answer is mentioned in the context:

"The average win rate of GPT-4 and human translations to prevent annotators from identifying GPT-4. The average win rate of GPT is 15.5/40 (36.25%)."
